In [8]:
# Imports and set torch device
import pyvista as pv
import torch
import meshplot as mp
import kaolin as kal
import matplotlib.pyplot as plt
import trimesh
import numpy as np
import open3d as o3d
import open3d.core as o3c

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'bookshelf'
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


In [9]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices = mesh.vertices.cpu().numpy()
faces = mesh.faces.cpu().numpy()
colors = mesh.vertex_normals.cpu().numpy()
vertices = (vertices - np.mean(vertices, axis=0)) / np.std(vertices)
# Sample Point Cloud
np.random.seed(42)
n_samples = int(vertices.shape[0] * 2.)
trimeshMesh = trimesh.Trimesh(vertices, faces)
point_cloud = np.asarray(trimesh.sample.sample_surface_even(trimeshMesh, n_samples)[0])
# point_cloud = (point_cloud - np.mean(point_cloud, axis=0)) / np.std(point_cloud)
print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])
print('Number of sampled points:', point_cloud.shape[0])
mp.plot(point_cloud, shading={'point_size':0.2})
plt.show()

Number of vertices:  5786
Number of faces:  8624
Number of sampled points: 11572


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.003204…

## Using Alpha Shapes

In [ ]:
alpha = 0.15
o3d_points = o3d.cpu.pybind.utility.Vector3dVector(point_cloud)
pcd = o3d.cpu.pybind.geometry.PointCloud(o3d_points)
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
remeshed_vertices = np.asarray(mesh.vertices)
remeshed_faces = np.asarray(mesh.triangles)
print('Number of remeshed vertices: ', remeshed_vertices.shape[0])
print('Number of remeshed faces: ', remeshed_faces.shape[0])
mp.plot(remeshed_vertices, remeshed_faces, np.random.rand(remeshed_faces.shape[0]))
plt.show()

## Using Ball Pivoting